<a href="https://colab.research.google.com/github/SohamSen21/Recommender-System-meets-Mechanism-Design/blob/main/Matrix_Factorization_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

def mf(R, k, n_epoch=5000, lr=.0003, l2=.04):
  tol = .001  # Tolerant loss.
  m, n = R.shape
  # Initialize the embedding weights.
  P = np.random.rand(m, k)
  Q = np.random.rand(n, k)
  for epoch in range(n_epoch):
    # Update weights by gradients.
    for u, i in zip(*R.nonzero()):
      err_ui = R[u,i] - P[u,:].dot(Q[i,:])
      for j in range(k):
        P[u][j] += lr * (2 * err_ui * Q[i][j] - l2/2 * P[u][j])
        Q[i][j] += lr * (2 * err_ui * P[u][j] - l2/2 * Q[i][j])
    # compute the loss.
    E = (R - P.dot(Q.T))**2
    obj = E[R.nonzero()].sum() + lr*((P**2).sum() +(Q**2).sum())
    if obj < tol:
        break
  return P, Q

In [2]:
np.random.seed(777)

# Make missing more prevail.
stars = np.arange(6)
p = np.array([10, 1, 1, 1, 1, 1])
m = 5
n = 10

# A 5-star rating matrix.
ratings = np.random.choice(stars, size=m*n, p=p / p.sum()).reshape((m, n))
print(ratings)

[[0 0 0 0 3 4 1 2 0 0]
 [0 0 0 0 5 0 1 0 0 0]
 [0 0 0 0 0 4 0 0 0 3]
 [0 0 0 0 0 0 0 4 2 0]
 [0 0 1 0 2 0 0 0 0 2]]


In [3]:
P, Q = mf(ratings, k=3)

print(P)  # User embeddings.

[[0.63030098 1.32632817 0.22731696]
 [0.91776555 1.65371568 1.13847576]
 [0.94394792 1.20334036 0.08460967]
 [1.48581284 1.84234102 0.85833273]
 [0.90108838 0.48804721 0.31216772]]


In [4]:
print(Q)  # Item embeddings.

[[ 0.69258857  0.83594341  0.42432199]
 [ 0.8487743   0.54679121  0.35410346]
 [ 0.73827766  0.1010681   0.87686572]
 [ 0.33625828  0.89183268  0.296849  ]
 [ 0.87169152  1.62740441  1.29970871]
 [ 1.50389245  2.12570844  0.65550343]
 [ 0.37660684  0.52827304 -0.1436639 ]
 [ 1.11657344  0.99507904  0.50060044]
 [ 0.32235812  0.39279822  0.93554762]
 [ 1.48046418  1.27550157  0.17612006]]


In [5]:
# User similarity.
l2 = np.sqrt(pow(P, 2).sum(axis=1))
user_sim = P.dot(P.T) / np.outer(l2, l2)
print(np.round(user_sim, 2))

[[1.   0.92 0.97 0.96 0.81]
 [0.92 1.   0.87 0.97 0.84]
 [0.97 0.87 1.   0.96 0.89]
 [0.96 0.97 0.96 1.   0.93]
 [0.81 0.84 0.89 0.93 1.  ]]


In [6]:
# Not run.
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(P)

array([[1.        , 0.9238834 , 0.97105358, 0.95563301, 0.80800464],
       [0.9238834 , 1.        , 0.8732077 , 0.96936203, 0.84127185],
       [0.97105358, 0.8732077 , 1.        , 0.95740149, 0.89236845],
       [0.95563301, 0.96936203, 0.95740149, 1.        , 0.92913622],
       [0.80800464, 0.84127185, 0.89236845, 0.92913622, 1.        ]])

In [7]:
predictions = P.dot(Q.T)
mask = np.zeros_like(ratings)
mask[ratings.nonzero()] = 1

# Mask out unknown ratings as 0 for ease of comparison.
print(np.round(predictions * mask, 2))

[[0.   0.   0.   0.   3.   3.92 0.91 2.14 0.   0.  ]
 [0.   0.   0.   0.   4.97 0.   1.06 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   4.03 0.   0.   0.   2.95]
 [0.   0.   0.   0.   0.   0.   0.   3.92 2.01 0.  ]
 [0.   0.   0.99 0.   1.99 0.   0.   0.   0.   2.01]]


In [8]:
# Mask out known ratings as 0 for ease of comparison.
print(np.round(predictions * (1 - mask), 2))

[[1.64 1.34 0.8  1.46 0.   0.   0.   0.   0.94 2.66]
 [2.5  2.09 1.84 2.12 0.   5.64 0.   3.24 2.01 3.67]
 [1.7  1.49 0.89 1.42 2.89 0.   0.98 2.29 0.86 0.  ]
 [2.93 2.57 2.04 2.4  5.41 6.71 1.41 0.   0.   4.7 ]
 [1.16 1.14 0.   0.83 0.   2.6  0.55 1.65 0.77 0.  ]]
